In [8]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import zipfile
from keras.metrics import Precision,Recall

In [2]:
with zipfile.ZipFile("/content/Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset")

In [3]:
test_data = image_dataset_from_directory("/content/Dataset/Dataset/Test",image_size = (224,224),batch_size = 128*2)


Found 80362 files belonging to 2 classes.


In [4]:
train_data = image_dataset_from_directory("/content/Dataset/Dataset/Train",image_size = (224,224),batch_size = 50)


Found 120542 files belonging to 2 classes.


In [5]:
conv_base = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet'
)
conv_base.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [9]:
inputs = keras.Input(shape = (224,224,3))
x = keras.applications.resnet50.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256,activation = "relu")(x)
x = layers.Dense(128,activation = "relu")(x)
x  =layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model  = keras.Model(inputs = inputs,outputs = outputs)

In [10]:
model.compile(optimizer="adam" ,loss ="binary_crossentropy",metrics =["accuracy", Precision(name='precision'), Recall(name='recall')])

In [11]:
model.fit(train_data,epochs = 10)


Epoch 1/10
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 77s 25ms/step - accuracy: 0.8561 - loss: 0.3746 - precision: 0.6949 - recall: 0.5764
Epoch 2/10
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 22ms/step - accuracy: 0.8706 - loss: 0.3321 - precision: 0.7582 - recall: 0.5551
Epoch 3/10
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 55s 23ms/step - accuracy: 0.8717 - loss: 0.3290 - precision: 0.7621 - recall: 0.5582
Epoch 4/10
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 22ms/step - accuracy: 0.8738 - loss: 0.3262 - precision: 0.7687 - recall: 0.5644
Epoch 5/10
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 22ms/step - accuracy: 0.8749 - loss: 0.3245 - precision: 0.7699 - recall: 0.5694
Epoch 6/10
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 22ms/step - accuracy: 0.8752 - loss: 0.3226 - precision: 0.7708 - recall: 0.5700
Epoch 7/10
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 22ms/step - accuracy: 0.8762 - loss: 0.3198 - precision: 0.7740 - recall: 0.5724
Epoch 8/10
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 22ms/step - accuracy: 0.8775 - loss: 0.3182 - precision: 0.7811 -

In [12]:
all_labels = []
for i,l in test_data:
  all_labels.extend(l.numpy())

In [13]:
clean_labels = [int(x) for x in all_labels]

In [14]:
y_pred = model.predict(test_data)

314/314 ━━━━━━━━━━━━━━━━━━━━ 41s 103ms/step


In [15]:
y_pred_labels = (y_pred > 0.5).astype(int).flatten()


In [16]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score

In [17]:
accuracy_score(clean_labels,y_pred_labels)

0.7023468803663423

In [18]:
recall_score(clean_labels,y_pred_labels)

0.1534090909090909

In [19]:
precision_score(clean_labels,y_pred_labels)

0.20802919708029197

In [20]:
f1_score(clean_labels,y_pred_labels)

0.176592082616179

In [21]:
confusion_matrix(clean_labels,y_pred_labels)

array([[53877,  9765],
       [14155,  2565]])